In [ ]:
%load_ext autoreload
%autoreload 2

In [207]:
from utils.DBUpdater import DBUpdater

In [208]:
dbup = DBUpdater()
dbup.update()

# Data Formatting

In [ ]:
from scipy.sparse import spmatrix
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from datetime import timedelta
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import json
import os
load_dotenv()

In [210]:
from modules.database_tables.Orders import Orders
from modules.database_tables.Products import Products
from modules.database_tables.TaskCards import TaskCards

In [211]:
username = os.getenv("mongodb_user")
password = os.getenv("mongodb_password")

In [212]:
orders_table = Orders(username, password)
products_table = Products(username, password)
taskcards_table = TaskCards(username, password)

In [213]:
original_data = []
for order in orders_table.get_orders():
    product = products_table.find({"order_id": order["_id"]})
    tasks = taskcards_table.find_many({"product_id": product["_id"]})
    order["product"] = product
    order["product"]["task_cards"] = tasks
    
    order.pop("_id")
    order['delivery_date'] = order['delivery_date'].strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    order['order_created_at'] = order['order_created_at'].strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    order['product'].pop("_id")
    order['product'].pop("order_id")
    
    for task in order['product']['task_cards']:
        task.pop("_id")
        task.pop("product_id")
        task['task_time_start'] = task['task_time_start'].strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        task['task_time_end'] = task['task_time_end'].strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    original_data.append(order)
    
json_data = json.dumps(original_data, indent=4)
with open("output.json", "w") as f:
    f.write(json_data)

Make splitted between days tasks one whole task_card

In [ ]:
# data = original_data.copy()
# for order in data:
#     tasks: list[dict] = order['product']['task_cards']
#     task_dupe_data = {}
#     for task in tasks:
#         sort_order = task['sort_order']
#         if sort_order not in task_dupe_data:
#             task_dupe_data[sort_order] = []
#         task_dupe_data[sort_order].append(task)
    
#     task_dupe_data = {k: v for k, v in task_dupe_data.items() if len(v) > 1}
#     for sort_order, duplicates in task_dupe_data.items():
#         first_instance = duplicates[0]
#         total_duration = sum(task['task_duration'] for task in duplicates)
#         first_instance['task_duration'] = total_duration
#         task_time_end = (datetime.strptime(first_instance['task_time_start'], "%Y-%m-%dT%H:%M:%S.%fZ") + timedelta(seconds=total_duration)).strftime("%Y-%m-%dT%H:%M:%S.%fZ")
#         first_instance['task_time_end'] = task_time_end
        
#         # Remove all duplicates from the tasks list
#         for duplicate in duplicates:
#             order['product']['task_cards'].remove(duplicate)
        
#         # Find the correct position to insert the merged task
#         insert_index = 0
#         for i, task in enumerate(order['product']['task_cards']):
#             if task['sort_order'] > first_instance['sort_order']:
#                 insert_index = i
#                 break
#         else:
#             insert_index = len(order['product']['task_cards'])
        

#         # Insert the merged task
#         order['product']['task_cards'].insert(insert_index, first_instance)
        
# json_data = json.dumps(data, indent=4)
# with open("output_deduped.json", "w") as f:
#     f.write(json_data)


In [214]:
def combine_duped_taskcards(taskcards: list[dict]) -> list[dict]:
    task_dupe_data = {}
    for task in tasks:
        sort_order = task['sort_order']
        if sort_order not in task_dupe_data:
            task_dupe_data[sort_order] = []
        task_dupe_data[sort_order].append(task)
    
    task_dupe_data = {k: v for k, v in task_dupe_data.items() if len(v) > 1}
    for sort_order, duplicates in task_dupe_data.items():
        first_instance = duplicates[0]
        total_duration = sum(task['task_duration'] for task in duplicates)
        first_instance['task_duration'] = total_duration
        task_time_end = (datetime.strptime(first_instance['task_time_start'], "%Y-%m-%dT%H:%M:%S.%fZ") + timedelta(seconds=total_duration)).strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        first_instance['task_time_end'] = task_time_end
        
        # Remove all duplicates from the tasks list
        for duplicate in duplicates:
            order['product']['task_cards'].remove(duplicate)
        
        # Find the correct position to insert the merged task
        insert_index = 0
        for i, task in enumerate(order['product']['task_cards']):
            if task['sort_order'] > first_instance['sort_order']:
                insert_index = i
                break
        else:
            insert_index = len(order['product']['task_cards'])
        

        # Insert the merged task
        taskcards.insert(insert_index, first_instance)
        
    return taskcards

In [215]:
data = original_data.copy()
for order in data:
    tasks: list[dict] = order['product']['task_cards']
    new_tasks: list[dict] = combine_duped_taskcards(tasks)
    # order['product']['task_cards'] = new_tasks
    
json_data = json.dumps(data, indent=4)
with open("output_deduped.json", "w") as f:
    f.write(json_data)